## Inserting food into knowledge graph
- kg : nodes, edges -> compound
smiles 고유값 10556개 (Compound::DB로 필터링),
smiles 고유값 1706개 (type == 0으로 필터링) -> 이걸시용해야할듯


- smiles : compound smiles
smiles 고유값 1706개 => id값으로 kg node랑 mapping됨 (0번부터 1705번)


- foodb : food - compound relation,
smiles 고유값 1589개


- db_smiles : drugbank smiles data
smiles 고유값 8807개 -> id - smiles 교차검증용

In [31]:
import pandas as pd
import numpy as np
from pathlib import Path

# data load
nodes = pd.read_csv("data/DRKG+DrugBank/nodes.tsv", sep="\t", names=["id", "name", "type"])
edges = pd.read_csv("data/DRKG+DrugBank/edges.tsv", sep="\t", names=["h", "r", "t"])
smiles = pd.read_csv('data/DRKG+DrugBank/smiles.tsv', sep="\t", names=["id", "smiles"])
foodb = pd.read_excel("data/dummy/foodb_final.xlsx")
db_smiles = pd.read_csv("data/dummy/drugbank_smiles.txt", sep="\t", header=None, names=["drug_id", "smiles"])

# preprocesing
foodb = foodb[['orig_food_common_name', 'moldb_smiles', 'orig_content']].rename(columns={'orig_food_common_name': 'food_name', 'moldb_smiles' : 'smiles', 'orig_content' : 'edge_weight'})
foodb

,food_name,smiles,edge_weight
0,Cabbage,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,100.0
1,Kiwi fruit,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,800.0
2,Kiwifruit,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,520.0
3,Kiwifruit,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,560.0
4,Onion,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,300.0
...,...,...,...
433457,Milk,CCCCCCCCCCCC(O)=O,77.0
433458,Milk,CC\C=C/C\C=C/C\C=C/C\C=C/C\C=C/C\C=C/CCC(O)=O,0.0
433459,Milk,CC\C=C/C\C=C/C\C=C\C\C=C/C\C=C/CCCC(O)=O,0.0
433460,Milk,CCCCCCCCCCCCCCC(O)=O,0.0


In [32]:
# dataset cross validation
# 1706개 전부 동일한 값으로 mapping됨
temp_node = nodes.copy()
temp_node['compound'] = temp_node['name'].str.split("::").str[-1]
temp = pd.merge(temp_node, db_smiles, left_on=['compound'], right_on=['drug_id'] ,how='left')
df = pd.merge(temp, smiles, on=['id'], how='left')
only_compound = df[df['type']==0]
only_compound

,id,name,type,compound,drug_id,smiles_x,smiles_y
0,0,Compound::DB00006,0,DB00006,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,1,Compound::DB00014,0,DB00014,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,2,Compound::DB00027,0,DB00027,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,3,Compound::DB00035,0,DB00035,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,4,Compound::DB00080,0,DB00080,DB00080,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...
...,...,...,...,...,...,...,...
1701,1701,Compound::DB13873,0,DB13873,DB13873,CC(C)(OC1=CC=C(C=C1)C(=O)C1=CC=C(Cl)C=C1)C(O)=O,CC(C)(OC1=CC=C(C=C1)C(=O)C1=CC=C(Cl)C=C1)C(O)=O
1702,1702,Compound::DB13874,0,DB13874,DB13874,CC(C)(O)CNC1=NC(=NC(NC2=CC(=NC=C2)C(F)(F)F)=N1...,CC(C)(O)CNC1=NC(=NC(NC2=CC(=NC=C2)C(F)(F)F)=N1...
1703,1703,Compound::DB13878,0,DB13878,DB13878,CO[C@H](C)[C@H](NC(=O)OC)C(=O)N1CCC[C@H]1C1=NC...,CO[C@H](C)[C@H](NC(=O)OC)C(=O)N1CCC[C@H]1C1=NC...
1704,1704,Compound::DB13879,0,DB13879,DB13879,CC(C)(C)[C@@H]1NC(=O)O[C@@H]2CCC[C@H]2OC\C=C\C...,[H][C@@]12CN(C(=O)[C@@]([H])(NC(=O)O[C@]3([H])...


In [35]:
##  food-compound mapping
food_compound = pd.merge(foodb, only_compound, left_on=['smiles'], right_on=['smiles_x'], how='left')
food_compound['name'].value_counts().__len__() # 연결되는 smiles는 39종류
temp = food_compound[food_compound['compound'].notna()] 
temp['food_name'].value_counts().__len__() # smiles와 연결되는 food는 1500종류
fd_connect = temp.drop_duplicates().drop(columns=['type', 'smiles_x', 'smiles_y', 'drug_id']) # food - smiles interaction은 21604개가 발생함
fd_connect['id'] = fd_connect['id'].astype('Int64')
fd_connect.rename(columns={'id': 'drug_id', },  inplace=True)

# food_id indexing
food_id_start = 97243 # node의 id 97243부터 food로 지정
food_name = fd_connect['food_name'].unique()
food_id_map = {food_name: i for i, food_name in enumerate(food_name, start=food_id_start)}
fd_connect['food_id'] = fd_connect['food_name'].map(food_id_map)
cols = ['food_id'] + [col for col in fd_connect.columns if col != 'food_id']
fd_connect = fd_connect[cols]
fd_connect.to_csv('fd_connect.csv', index=False)
fd_connect


,food_id,food_name,smiles,edge_weight,drug_id,name,compound
19954,97243,Alfalfa seeds,CCO,0.00000,661,Compound::DB00898,DB00898
19959,97243,Alfalfa seeds,CC(C)[C@@H](C)\C=C\[C@@H](C)[C@@]1([H])CC[C@@]...,0.00000,19,Compound::DB00153,DB00153
19960,97243,Alfalfa seeds,CC(C)CCC[C@@H](C)CCC[C@@H](C)CCC[C@]1(C)CCC2=C...,0.00000,22,Compound::DB00163,DB00163
19963,97243,Alfalfa seeds,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,0.58000,14,Compound::DB00140,DB00140
19964,97243,Alfalfa seeds,OC(=O)C1=CN=CC=C1,1.80000,422,Compound::DB00627,DB00627
...,...,...,...,...,...,...,...
425958,97297,Cabbage,N[C@@H](CCC(O)=O)C(O)=O,1939.95000,15,Compound::DB00142,DB00142
425962,97297,Cabbage,CSCC[C@H](N)C(O)=O,86.20000,12,Compound::DB00134,DB00134
425968,97297,Cabbage,[Se],0.01265,1630,Compound::DB11135,DB11135
425970,97297,Cabbage,NC1=NC(=O)C2=NC(CNC3=CC=C(C=C3)C(=O)N[C@@H](CC...,0.47250,20,Compound::DB00158,DB00158


In [ ]:
# making new kg nodes (type = 14)
food_nodes = fd_connect[['food_id', 'food_name']].reset_index(drop=True)
food_nodes['food_name'] = 'Food::' + food_nodes['food_name']
food_nodes.rename(columns={'food_id' : 'id', 'food_name' : 'name'}, inplace=True)
food_nodes['type'] = 14
food_nodes = food_nodes.drop_duplicates()

kg_nodes = pd.concat([nodes, food_nodes], ignore_index=True)
kg_nodes.to_csv('new_kg_nodes.tsv', sep='\t', index=False, header=False)
kg_nodes

,id,name,type
0,0,Compound::DB00006,0
1,1,Compound::DB00014,0
2,2,Compound::DB00027,0
3,3,Compound::DB00035,0
4,4,Compound::DB00080,0
...,...,...,...
98738,98738,Food::Brussel-Sprout,14
98739,98739,Food::Cantaloupe,14
98740,98740,Food::Cornmint,14
98741,98741,Food::Olive,14


In [154]:
# making new kg edges (type = 107)
fd_connect['relationship'] = 107
food_edges = fd_connect[['food_id', 'relationship', 'drug_id']].reset_index(drop=True)
food_edges.rename(columns={'food_id':'h', 'relationship':'r', 'drug_id':'t'}, inplace=True)

kg_edges = pd.concat([edges, food_edges], ignore_index=True)
kg_edges.to_csv('new_kg_edges.tsv', sep='\t', index=False, header=False)
kg_edges

,h,r,t
0,57548,105,57548
1,57548,105,70486
2,57548,105,57550
3,57548,105,64179
4,57548,105,62588
...,...,...,...
5741273,98742,107,12
5741274,98742,107,1630
5741275,98742,107,20
5741276,98742,107,1114
